Code majorly copied from https://github.com/tomasonjo/blogs/blob/master/llm/openaifunction_constructing_graph.ipynb
Author's medium article: https://medium.com/@bratanic-tomaz/constructing-knowledge-graphs-from-text-using-openai-functions-096a6d010c17

In [78]:
from dotenv import load_dotenv
import os

load_dotenv()
# OPENAI_API_KEY in .env file

True

In [7]:
from langchain.graphs import Neo4jGraph

url = "bolt://localhost:7687"
username ="neo4j"
password = "athena_password"
graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

In [9]:
from typing import List, Dict, Any, Optional

from langchain.graphs.graph_document import (
    Node as BaseNode,
    Relationship as BaseRelationship,
    GraphDocument
)
from langchain.pydantic_v1 import Field, BaseModel
from langchain.schema import Document


class Property(BaseModel):
    """A single property consisting of key and value"""
    
    key: str = Field(..., description="key")
    value: str = Field(..., description="value")

class Node(BaseNode):
    properties: Optional[List[Property]] = Field(None, description="List of node properties")

class Relationship(BaseRelationship):
    properties: Optional[List[Property]] = Field(None, description="List of relationship properties")

class KnowledgeGraph(BaseModel):
    """Generate a knowledge graph with entities and relationships."""

    nodes: List[Node] = Field(..., description="List of nodes in the knowledge graph")
    rels: List[Relationship] = Field(..., description="List of relationships in the knowledge graph")

In [10]:
def format_property_key(s: str) -> str:
    words = s.split()

    if not words:
        return s

    first_word = words[0].lower()
    capitalized_words = [word.capitalize() for word in words[1:]]

    return "".join([first_word] + capitalized_words)

def props_to_dict(props) -> dict:
    """Convert properties to a dictionary."""

    properties = {}

    if not props:
        return properties

    for p in props:
        properties[format_property_key(p.key)] = p.value

    return properties

def map_to_base_node(node: Node) -> BaseNode:
    """Map the KnowledgeGraph Node to the base Node."""

    properties = props_to_dict(node.properties) if node.properties else {}
    # Add name property for better Cypher statement generation
    properties["name"] = node.id.title()

    return BaseNode(id=node.id.title(), type=node.type.capitalize(), properties=properties)


def map_to_base_relationship(rel: Relationship) -> BaseRelationship:
    """Map the KnowledgeGraph Relationship to the base Relationship."""

    source = map_to_base_node(rel.source)
    target = map_to_base_node(rel.target)
    properties = props_to_dict(rel.properties) if rel.properties else {}

    return BaseRelationship(source=source, target=target, type=rel.type, properties=properties)

In [113]:
from langchain.chains.openai_functions import create_structured_output_chain
from langchain.chains.openai_functions.extraction import create_extraction_chain_pydantic
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

gpt3 = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)
gpt4 = ChatOpenAI(model="gpt-4", temperature=0)

In [114]:
def get_extraction_chain(
    allowed_nodes: Optional[List[str]] = None,
    allowed_rels: Optional[List[str]] = None
    ):
    prompt = ChatPromptTemplate.from_messages(
        [(
          "system",
          f"""# Knowledge Graph Instructions for GPT-4
## 1. Overview
You are a top-tier algorithm designed for extracting information from markdown notes in structured formats to build a knowledge graph.
- **Nodes** represent entities and concepts. They're akin to Wikipedia nodes.
- The aim is to achieve simplicity and clarity in the knowledge graph, making it accessible for a vast audience.
## 2. Labeling Nodes
- **Consistency**: Ensure you use basic or elementary types for node labels.
  - For example, when you identify an entity representing a person, always label it as **"person"**. Avoid using more specific terms like "mathematician" or "scientist".
- **Node IDs**: Never utilize integers as node IDs. Node IDs should be names or human-readable identifiers found in the text.
{'- **Allowed Node Labels:**' + ", ".join(allowed_nodes) if allowed_nodes else ""}
{'- **Allowed Relationship Types**:' + ", ".join(allowed_rels) if allowed_rels else ""}
## 3. Handling Numerical Data and Dates
- Numerical data, like age or other related information, should be incorporated as attributes or properties of the respective nodes.
- **No Separate Nodes for Dates/Numbers**: Do not create separate nodes for dates or numerical values. Always attach them as attributes or properties of nodes.
- **Property Format**: Properties must be in a key-value format.
- **Quotation Marks**: Never use escaped single or double quotes within property values.
- **Naming Convention**: Use camelCase for property keys, e.g., `birthDate`.
## 4. Coreference Resolution
- **Maintain Entity Consistency**: When extracting entities, it's vital to ensure consistency.
If an entity, such as "John Doe", is mentioned multiple times in the text but is referred to by different names or pronouns (e.g., "Joe", "he"),
always use the most complete identifier for that entity throughout the knowledge graph. In this example, use "John Doe" as the entity ID.
Remember, the knowledge graph should be coherent and easily understandable, so maintaining consistency in entity references is crucial.
## 5. Strict Compliance
Adhere to the rules strictly. Non-compliance will result in termination.
          """),
            ("human", "Use the given format to extract information from the following input: {input}"),
            ("human", "Tip: Make sure to answer in the correct format"),
        ])
    return create_structured_output_chain(KnowledgeGraph, gpt4, prompt, verbose=False)
    # apparently create_extraction_chain_pydantic is better / uses less tokens according to https://github.com/langchain-ai/langchain/issues/7935
    # return create_extraction_chain_pydantic(KnowledgeGraph, llm, prompt, verbose=True)

In [85]:
def extract_and_store_graph(
  document: Document,
  nodes:Optional[List[str]] = None,
  rels:Optional[List[str]] = None
  ) -> None:

  # Extract graph data using OpenAI functions
  extract_chain = get_extraction_chain(nodes, rels)
  data = extract_chain.run(document.page_content)

  # Construct a graph document
  graph_document = GraphDocument(
    nodes = [map_to_base_node(node) for node in data.nodes],
    relationships = [map_to_base_relationship(rel) for rel in data.rels],
    source = document
  )

  # Store information into a graph
  graph.add_graph_documents([graph_document])

In [86]:
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import TokenTextSplitter


loader = DirectoryLoader('../../../../', glob="**/*.md")
# markdown_path = "../../../../09-01-23 Readings.md"
# loader = UnstructuredMarkdownLoader(markdown_path)
raw_docs = loader.load()
text_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=24)

docs = text_splitter.split_documents(raw_docs)

In [87]:
docs

[Document(page_content='Heinrich Schliemann\ndiscovers Troy in the 1870s - it’s now known to be a real place! Archaeology is having a revolution. Period of romanticism and philhellenism - Greece gaining independence from Ottoman empire.\n\nfound several layers of different rebuildings of Troy\nTroy - very strategic location at mouth of water trade route\n\nvery destructive digging and some looting - dug a giant hole\n\n“The Treasure of Priam” - lots of wealth - gold & copper items\n\nbro liked to lie, was brilliant linguist but generally questionable & problematic\n\nFueled by success of Troy, Schliemann finds Mycenae\n- again digs straight down, into burial sites “Grave Circle A”\n- finds again lots of gold - remember no gold found in Greece, had to have come from somewhere else via war / trade\n- “palace of Agamemnon”\n\nStill scholarship being conducted on these findings with new information, techniques, and technology. DNA analysis\n\nMinoans 1700 - 1500 BCE\n- don’t know what lang

In [88]:
from tqdm import tqdm

for i, d in tqdm(enumerate(docs), total=len(docs)):
    extract_and_store_graph(d)

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [11:01<00:00, 220.43s/it]


In [76]:
# Delete the graph
graph.query("MATCH (n) DETACH DELETE n")

[]

In [100]:
# want to try to just grab the entitites from the text
class EntityList(BaseModel):
    """A list of entities"""
    
    entities: Optional[List[str]] = Field(None, description="List of the names of entities")

In [115]:
def get_entity_extraction_chain():
    prompt = ChatPromptTemplate.from_messages(
        [(
          "system",
          f"""# Entity identification and resolution instructions for GPT-4
## 1. Overview
You are a top-tier algorithm designed for extracting entities from markdown notes.
- **Entities** represent nouns of interest. Entities can be people, places, things, events, and ideas.
## 3. Handling Numerical Data and Dates
- Numerical data, like age or other related information, should not be identified as entities
## 4. Coreference Resolution
- **Maintain Entity Consistency**: When extracting entities, it's vital to ensure consistency.
If an entity, such as "John Doe", is mentioned multiple times in the text but is referred to by different names or pronouns (e.g., "Joe", "he"),
always use the most complete identifier for that entity throughout the knowledge graph. In this example, use "John Doe" as the entity ID.
Remember, the knowledge graph should be coherent and easily understandable, so maintaining consistency in entity references is crucial.
## 5. Strict Compliance
Adhere to the rules strictly. Non-compliance will result in termination.
          """),
            ("human", "Use the given format to extract information from the following input: {input}"),
            ("human", "Tip: Make sure to answer in the correct format"),
        ])
    return create_structured_output_chain(EntityList, gpt4, prompt, verbose=False)

In [103]:
def extract_entities(document: Document) -> List[Entity]:

  # Extract entity data using OpenAI functions
  extract_chain = get_entity_extraction_chain()
  data = extract_chain.run(document.page_content)
  
  return data

In [116]:
# extract entities
entity_lists = []

# runs in 20 seconds for gpt3.5, 1 minute for gpt4
for i, d in tqdm(enumerate(docs), total=len(docs)):
    entity_lists.append(extract_entities(d))

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [01:06<00:00, 22.25s/it]


In [117]:
entity_lists

[EntityList(entities=['Heinrich Schliemann', 'Troy', 'Archaeology', 'Period of romanticism', 'Philhellenism', 'Greece', 'Ottoman empire', 'The Treasure of Priam', 'Gold', 'Copper items', 'Mycenae', 'Grave Circle A', 'Palace of Agamemnon', 'DNA analysis', 'Minoans', 'Greek legend Minos', 'Crete', 'Egypt', 'Cyprus', 'Italy', 'Minoan Palaces', 'Phaistos', 'Knossos', 'Malia', 'Zakros', 'Bournia', 'Kommos', 'Redistributed economy', 'Cretan hieroglyphic script', 'Phaistos hieroglyphic script', 'Linear A', 'Aegean', 'Akrotiri', 'Volcanic eruption', 'Mycenaeans', 'Mycenae', 'Bronze age', 'Neolithic times', 'Tin', 'Bronze', 'Textiles', 'Olive oil', 'Perfume', 'Stone architecture', 'Beehive tomb', 'Tholos', 'Linear B', 'Anatolia', 'Kingdom of Pylos']),
 EntityList(entities=['Mycenaean Sparta', 'Ayios Vassileios', 'Spartan palace', 'Linear B script tablets', 'administrative center in Mycenaean culture', 'imported & otherwise high value items', 'Linear B', 'Linear A', 'Minoan language', 'Knossos',

In [110]:
merged_gpt3 = entity_lists[0].entities + entity_lists[1].entities + entity_lists[2].entities

In [118]:
merged_gpt4 = entity_lists[0].entities + entity_lists[1].entities + entity_lists[2].entities

In [131]:
deduped_gpt3 = set(merged_gpt3)
print(len(deduped_gpt3))
deduped_gpt4 = set(merged_gpt4)
print(len(deduped_gpt4))

intersection = deduped_gpt3.intersection(deduped_gpt4)
print(len(intersection))

diff = deduped_gpt3.symmetric_difference(deduped_gpt4)

print(f"GPT 3: {sorted(list(deduped_gpt3))}")
print(f"GPT 4: {sorted(list(deduped_gpt4))}")
print(f"Intersection: {sorted(list(intersection))}")
print(f"Diff: {sorted(list(diff))}")

99
94
50
GPT 3: ['1100 BC', '1200 BC', 'Aegean', 'Akrotiri', 'Anatolia', 'Archaeology', 'Ayios Vassileios', 'Bournia', 'Cnossus', 'Cretan hieroglyphic script', 'Crete', 'Cyprus', 'DNA analysis', 'Egypt', 'Franchthi Cave', 'Grave Circle A', 'Greece', 'Heinrich Schliemann', 'Hittite empire', 'Hittites', "Homer's epics", 'Italy', 'Knossos', 'Kommos', 'Late Bronze Age', 'Lerna', 'Libyans', 'Linear A', 'Linear B', 'Malia', 'Minoan Civilization', 'Minoan Palaces', 'Minoan art', 'Minoan religion', 'Minoans', 'Mother Goddess', 'Mycenae', 'Mycenaean Greeks', 'Mycenaean Sparta', 'Mycenaeans', 'Neolithic times', 'Ottoman empire', 'Parian Marble', 'Phaistos', 'Phaistos hieroglyphic script', 'Philistines', 'Pylos', 'Sir Arthur Evans', 'Syria', 'The Treasure of Priam', 'Thera', 'Troy', 'Zakros', 'administrative', 'agricultural goods', 'beehive tomb', 'burial sites', 'chariots', 'collapse', 'copper items', 'digging', 'eastern Mediterranean', 'economic', 'exports', 'fire', 'gold', 'language', 'layers'